# Clinical Trial Optimization 

Je presente ici ma solution pour le probléme d'opimisation d'un trie de sujet pour un test clinique.

### Presentation du probléme

Cette compétition organisée par Ingenii portait sur l'optimisation du tri des sujets pour un essai clinique. La description complète du problème est disponible sur [la plateforme Aqora](https://aqora.io/competitions/ingenii-clinical-trial), qui héberge la compétition.

Le problème consiste à regrouper les sujets en fonction de certains paramètres lors d'un essai clinique. L'objectif est qu'à l'issue de la formation des groupes, les sujets d'un même groupe soient adaptés aux tests cliniques qu'ils devront subir.

### Formulation mathématique

Nous avons 100 sujets que nous souhaitons répartir en deux groupes $p = \{1, 2\}$ de 50 personnes chacun. Chaque sujet $i$ est caractérisé par 3 paramètres $\vec{w_i} = (w_{i1}, w_{i2}, w_{i3})$. Nous mesurons la qualité de ce partitionnement à l'aide de la grandeur suivante :

$$
d = \sum_{s=1}^{3} |\Delta\mu_s| + \rho \sum_{s=1}^{3} |\Delta\sigma_{ss}| + 2\rho \sum_{s=1}^{3} \sum_{s' = s+1}^{3} |\Delta\sigma_{ss'}|
$$

où :

$$
\Delta\mu_s = \frac{1}{n} \sum_{i=1}^{n} w_{is}(x_{i1} - x_{i2})
$$

et :

$$
\Delta\sigma_{ss'} = \frac{1}{n} \sum_{i=1}^{n} w_{is} w_{is'} (x_{i1} - x_{i2})
$$

$x_{ip}$ est une variable binaire qui vaut 1 si le sujet $i$ appartient au groupe $p$, et 0 sinon.

La grandeur $d$ est appelée la *discrepancy*. Pour un regroupement idéal, la *discrepancy* est minimale. Ce problème de regroupement se ramène donc à résoudre le problème d'optimisation suivant :

$$
\min_{x} d
$$

sous les contraintes :

$$
\sum_{i}x_{ip} = \frac{n}{2}, \quad \text{Chaque groupe contient le même nombre de sujets.}
$$

$$
x_{i1} + x_{i2} = 1, \quad \text{Un sujet ne peut appartenir qu'à un seul groupe après le regroupement.}
$$

$$
x_{12} = 0, \quad \text{Ceci pour éviter la redondance des solutions, car on obtient le même regroupement en intervertissant les groupes 1 et 2.}
$$

### Reformulation du problème

Dans sa formulation initiale, le problème semble nous orienter vers l'utilisation de l'algorithme QAOA. Cependant, cette formulation ne permet pas une implémentation directe de l'algorithme en raison de la présence de valeurs absolues dans la fonction objective. De plus, le nombre de variables impliquées est très élevé : on a effectivement $2n-1$ variables, soit 199 variables dans ce cas, ce qui nécessiterait au moins une machine de 199 qubits, ce qui n'est actuellement pas disponible.

* **Écriture sans valeurs absolues :**
$$ a = |x| \Rightarrow x \leq a \text{ et } -x \leq a$$

Nous posons alors $z_s = |\Delta\mu_s|$, $z_{ss} = |\Delta\sigma_{ss}|$, et $z_{ss'} = |\Delta\sigma_{ss'}|$. La nouvelle formulation devient :
$$
\min_{x, z} \sum_{s=1}^{3} z_s + \rho \sum_{s=1}^{3} z_{ss} + 2\rho \sum_{s=1}^{3} \sum_{s' = s+1}^{3} z_{ss'}
$$
sous les contraintes suivantes :
$$
\sum_{i} x_{ip} = \frac{n}{2}, \quad \text{chaque groupe contient le même nombre de sujets.}
$$
$$
x_{i1} + x_{i2} = 1, \quad \text{un sujet ne peut appartenir qu'à un seul groupe après le regroupement.}
$$
$$
x_{12} = 0, \quad \text{cela évite la redondance des solutions, car intervertir les groupes 1 et 2 donnerait le même regroupement.}
$$
$$
\Delta\mu_s \leq z_s \text{ et } -\Delta\mu_s \leq z_s
$$
$$
\Delta\sigma_{ss} \leq z_{ss} \text{ et } -\Delta\sigma_{ss} \leq z_{ss}
$$
$$
\Delta\sigma_{ss'} \leq z_{ss'} \text{ et } -\Delta\sigma_{ss'} \leq z_{ss'}
$$

* **Réduction du nombre de variables :**

En remplaçant la contrainte $x_{i1} + x_{i2} = 1$ dans le problème, nous réduisons de $n$ le nombre de variables. En ajoutant les variables $z$, nous obtenons finalement $n - 1 + 9$, soit $n + 8$ variables.

### Implémentation
À ce stade, le problème est bien défini pour être implémenté avec l'algorithme QAOA. La prochaine étape consiste à le transformer en un problème QUBO, une transformation bien connue et générique. Cette étape sera réalisée à l'aide d'une bibliothèque dédiée de Qiskit.

In [29]:
from qiskit_optimization import QuadraticProgram

In [30]:
n =  100 #number of subjects
N = 1 # on enléve la division par 1 dans la fonction objective
rho = 0.5

In [31]:
import csv

data_list_csv = []
with open('pbc.csv', mode='r') as file:
    reader = csv.reader(file)
    # Skip the header
    next(reader)
    # Iterate over the rows and add them to the list
    for row in reader:
        # Convert strings to float for numeric columns
        data_list_csv.append([float(row[0]), float(row[1]), float(row[2])])

In [32]:
mod = QuadraticProgram("Clinical trial Optimization")

for i in range(1 , n + 1):
    var_name = f"x_{i}1"
    mod.binary_var(name=var_name)

# Variables continues   
W = []
for i in range(3):
    w_list = [row[i] for row in data_list_csv]
    w_sum = 0
    for w in w_list:
        w_sum = w_sum + w
    w_sum = w_sum/N
    W.append(w_sum)


mod.continuous_var(name = "z_1" , lowerbound=0 , upperbound=int(W[0]) + 1)
mod.continuous_var(name = "z_2" , lowerbound=0 , upperbound=int(W[1]) + 1)
mod.continuous_var(name = "z_3" , lowerbound=0 , upperbound=int(W[2]) + 1)

W_square = []
for i in range(3):
    w_list = [row[i] for row in data_list_csv]
    w_sum = 0
    for w in w_list:
        w_sum = w_sum + w**2
    w_sum = w_sum/N
    W_square.append(w_sum)

mod.continuous_var(name="z_11" , lowerbound=0 , upperbound=int(W_square[0]) + 1)
mod.continuous_var(name="z_22" , lowerbound=0 , upperbound=int(W_square[1]) + 1)
mod.continuous_var(name="z_33" , lowerbound=0 , upperbound=int(W_square[2]) + 1)


W_cross = []
for i in range(3):
    w_list_s = [row[i] for row in data_list_csv]
    for k in range(i + 1, 3):
        w_sum = 0
        w_list_s_prime = [row[k] for row in data_list_csv]
        for w_i, w_k in zip(w_list_s, w_list_s_prime):  
            w_sum = w_sum + w_i * w_k
        w_sum = w_sum / N
        W_cross.append(w_sum)
mod.continuous_var(name="z_12" , lowerbound=0 , upperbound=int(W_cross[0]) + 1)
mod.continuous_var(name="z_13" , lowerbound=0 , upperbound=int(W_cross[1]) + 1)
mod.continuous_var(name="z_23" , lowerbound=0 , upperbound=int(W_cross[2]) + 1)

#print(mod.prettyprint())

<Variable: 0 <= z_23 <= 30608403 (continuous)>

In [33]:
# definition de la fonction objective
mod.minimize(linear={"z_1": 1 ,"z_2": 1 , "z_3": 1 
                      , "z_11":0.5 , "z_22":0.5 , "z_33":0.5
                      , "z_12":1 , "z_13":1 , "z_23":1})

# Les contraintes

Contraite $ \sum_{i} x_{ip} = \frac{n}{2} \text{ avec } p = 1$

In [34]:
dict_som = {f"x_{i}1": 1 for i in range(1 , n + 1)}
mod.linear_constraint(dict_som , sense= "==" , rhs = n/2 , name = "repartition egale")

<LinearConstraint: x_1001 + x_101 + x_11 + x_111 + x_121 + x_131 + x_... == 50.0 'repartition egale'>

Contrainte: $x_{12} = 0 \Rightarrow x_{11} = 1$

In [35]:
mod.linear_constraint({"x_11": 1} , sense= "==" , rhs = 1 , name= "avoid redundance")

<LinearConstraint: x_11 == 1 'avoid redundance'>

Contrainte: $\Delta\mu_s \leq z_s \text{ et } -\Delta\mu_s \leq z_s$

$$
\Delta\mu_s \leq z_s \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}}{n}x_{i1} - z_{s} - \frac{1}{n}\sum_{i = 1}^{n}w_{is} \leq 0 \\

-\Delta\mu_s \leq z_s \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}}{n}x_{i1} + z_{s} - \frac{1}{n}\sum_{i = 1}^{n}w_{is} \geq 0 
$$



In [36]:
for i in range(3):
    dict_x_s = {}
    w_list = [row[i] for row in data_list_csv]
    dict_x_s = {f"x_{j + 1}1": (2*w_list[j])/N for j in range(0 , n)}
    dict_x_s.update({f"z_{i + 1}": -1})
    mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= W[i])


for i in range(3):
    dict_x_s = {}
    w_list = [row[i] for row in data_list_csv]
    dict_x_s = {f"x_{j + 1}1": (2*w_list[j])/N for j in range(0 , n)}
    dict_x_s.update({f"z_{i + 1}": 1})
    mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= W[i])

Contrainte: $\Delta\sigma_{ss} \leq z_{ss} \text{ et } -\Delta\sigma_{ss} \leq z_{ss}$

$$
\Delta\sigma_{ss} \leq z_{ss} \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}^2}{n}x_{i1} - z_{ss}  \leq \frac{1}{n}\sum_{i = 1}^{n}w_{is}^2 \\

- \Delta\sigma_{ss} \leq z_{ss} \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}^2}{n}x_{i1} + z_{ss}  \geq \frac{1}{n}\sum_{i = 1}^{n}w_{is}^2 
$$

In [37]:
for i in range(3):
    dict_x_s = {}
    w_list = [row[i] for row in data_list_csv]
    dict_x_s = {f"x_{j + 1}1": (2*(w_list[j]**2))/N for j in range(0 , n)}
    dict_x_s.update({f"z_{i + 1}" + f"{i + 1}": -1})
    mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= W_square[i])


for i in range(3):
    dict_x_s = {}
    w_list = [row[i] for row in data_list_csv]
    dict_x_s = {f"x_{j + 1}1": (2*(w_list[j]**2))/N for j in range(0 , n)}
    dict_x_s.update({f"z_{i + 1}" + f"{i + 1}": 1})
    mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= W_square[i])

Containte: $\Delta\sigma_{ss'} \leq z_{ss'} \text{ et } -\Delta\sigma_{ss'} \leq z_{ss'}$

$$
\Delta\sigma_{ss'} \leq z_{ss'} \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}w_{is'}}{n}x_{i1} - z_{ss'}  \leq \frac{1}{n}\sum_{i = 1}^{n}w_{is}w_{is'} \\

- \Delta\sigma_{ss'} \leq z_{ss'} \Rightarrow \sum_{i = 1}^{n}\frac{2w_{is}w_{is'}}{n}x_{i1} + z_{ss'}  \geq \frac{1}{n}\sum_{i = 1}^{n}w_{is}w_{is'} 
$$

In [38]:
for i in range(3):
    w_list_s = [row[i] for row in data_list_csv]
    for k in range(i + 1 , 3):
        dict_x_s = {}
        w_list_s_prime = [row[k] for row in data_list_csv]
        dict_x_s = {f"x_{j + 1}1": (2*(w_list_s[j]*w_list_s_prime[j]))/N for j in range(0 , n)}
        dict_x_s.update({f"z_{i + 1}" + f"{k + 1}": -1})
        mod.linear_constraint(linear=dict_x_s , sense= "<=" , rhs= W_cross[i])


for i in range(3):
    w_list_s = [row[i] for row in data_list_csv]
    for k in range(i + 1 , 3):
        dict_x_s = {}
        w_list_s_prime = [row[k] for row in data_list_csv]
        dict_x_s = {f"x_{j + 1}1": (2*(w_list_s[j]*w_list_s_prime[j]))/N for j in range(0 , n)}
        dict_x_s.update({f"z_{i + 1}" + f"{k + 1}": 1})
        mod.linear_constraint(linear=dict_x_s , sense= ">=" , rhs= W_cross[i])

In [39]:
print(mod.prettyprint())

Problem name: Clinical trial Optimization

Minimize
  z_1 + 0.5*z_11 + z_12 + z_13 + z_2 + 0.5*z_22 + z_23 + z_3 + 0.5*z_33

Subject to
  Linear constraints (20)
    x_1001 + x_101 + x_11 + x_111 + x_121 + x_131 + x_141 + x_151 + x_161
    + x_171 + x_181 + x_191 + x_201 + x_21 + x_211 + x_221 + x_231 + x_241
    + x_251 + x_261 + x_271 + x_281 + x_291 + x_301 + x_31 + x_311 + x_321
    + x_331 + x_341 + x_351 + x_361 + x_371 + x_381 + x_391 + x_401 + x_41
    + x_411 + x_421 + x_431 + x_441 + x_451 + x_461 + x_471 + x_481 + x_491
    + x_501 + x_51 + x_511 + x_521 + x_531 + x_541 + x_551 + x_561 + x_571
    + x_581 + x_591 + x_601 + x_61 + x_611 + x_621 + x_631 + x_641 + x_651
    + x_661 + x_671 + x_681 + x_691 + x_701 + x_71 + x_711 + x_721 + x_731
    + x_741 + x_751 + x_761 + x_771 + x_781 + x_791 + x_801 + x_81 + x_811
    + x_821 + x_831 + x_841 + x_851 + x_861 + x_871 + x_881 + x_891 + x_901
    + x_91 + x_911 + x_921 + x_931 + x_941 + x_951 + x_961 + x_971 + x_981
    + x_991 

# Transformation en QUBO

La conversion d'un problème en QUBO (Quadratic Unconstrained Binary Optimization) est un processus générique sur lequel je ne me suis pas attardé en profondeur, bien qu'il soit intéressant de s'intéresser à la manière la plus optimale de réaliser cette transformation. Dans ce travail, cette conversion sera effectuée à l'aide des outils proposés par Qiskit.

Avant la transformation, certaines mises en forme préliminaires sont nécessaires pour la formulation quadratique :
* Transformer les variables continues en variables discrètes.
* Convertir les contraintes d'inéquation en contraintes d'égalité.

In [40]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.converters import InequalityToEquality

# convert inequality to equality
mod_equality = InequalityToEquality().convert(mod)

In [41]:
print(mod_equality.prettyprint())

Problem name: Clinical trial Optimization

Minimize
  z_1 + 0.5*z_11 + z_12 + z_13 + z_2 + 0.5*z_22 + z_23 + z_3 + 0.5*z_33

Subject to
  Linear constraints (20)
    x_1001 + x_101 + x_11 + x_111 + x_121 + x_131 + x_141 + x_151 + x_161
    + x_171 + x_181 + x_191 + x_201 + x_21 + x_211 + x_221 + x_231 + x_241
    + x_251 + x_261 + x_271 + x_281 + x_291 + x_301 + x_31 + x_311 + x_321
    + x_331 + x_341 + x_351 + x_361 + x_371 + x_381 + x_391 + x_401 + x_41
    + x_411 + x_421 + x_431 + x_441 + x_451 + x_461 + x_471 + x_481 + x_491
    + x_501 + x_51 + x_511 + x_521 + x_531 + x_541 + x_551 + x_561 + x_571
    + x_581 + x_591 + x_601 + x_61 + x_611 + x_621 + x_631 + x_641 + x_651
    + x_661 + x_671 + x_681 + x_691 + x_701 + x_71 + x_711 + x_721 + x_731
    + x_741 + x_751 + x_761 + x_771 + x_781 + x_791 + x_801 + x_81 + x_811
    + x_821 + x_831 + x_841 + x_851 + x_861 + x_871 + x_881 + x_891 + x_901
    + x_91 + x_911 + x_921 + x_931 + x_941 + x_951 + x_961 + x_971 + x_981
    + x_991 

In [42]:
cto_qubo = QuadraticProgramToQubo().convert(mod_equality)
print(cto_qubo.prettyprint())

QiskitOptimizationError: 'Incompatible problem: Continuous variables are not supported! Can not convert inequality constraints to equality constraint because                     float coefficients are in constraints. '